In [13]:
from datetime import datetime
import pandas as pd
import plotly
import plotly.graph_objs as go
import simplejson as json
import ipywidgets as widgets
from datetime import timedelta
import dateutil.parser
from make_pydb import waitingHook


In [14]:
plotly.offline.init_notebook_mode(connected=True)

In [15]:
#helper method created to make different markers for diffferent types of event points (completed, initiated, not comp)
def make_scatter(startX, startY, successX, successY, stoppedX, stoppedY, motorColor, num, motor_name):
        start = go.Scatter(
            name = str(motor_name)+' Event Initiated',
            mode = 'markers',
            marker = dict(
            color = str(motorColor),
            size = 20,
            ),
            hoverinfo = 'x+y',
            x=startX,
            y=startY
        )
        stop = go.Scatter(
               name = 'Event Finished',
               mode = 'markers',
               marker = dict(
               color = 'black',
               size = 8
               ),
               hoverinfo = 'none',
               x=successX,
               y=successY
           )
        stopped = go.Scatter(
               name = 'Target Not Reached',
                mode = 'markers',
                marker = dict(
                color = 'red',
                size = 15
                ),
                hoverinfo = 'y',
                x=stoppedX,
                y=stoppedY
            )
        #only shows the legend for stopped and finished points if first time, prevents duplicate legend info
        if num>0:
            stopped.showlegend = False
            stop.showlegend = False
        if stoppedX!=None:
            data = [start, stop, stopped]
        else:
            data = [start, stop]
        return data
    
#called when an instrument is chosen from dropdown, the motor's df is found in the database for the given date range
#and appends points to lists sent to make_scatter to be given specific markers
def plot_motor(motor_name, time1, time2, num):
        colors = 2*(ord(motor_name[:1])),2*ord(motor_name[1:2]),2*(ord(motor_name[2:3]))
        motorColor = 'rgb'+str(colors)
        time1=datetime.strftime(time1, "%Y-%m-%d")+" 00:00:00"
        time2=datetime.strftime(time2, "%Y-%m-%d")+" 23:59:59"
        wh = waitingHook(str(motor_name))
        df = wh.get_data( str(motor_name), time1, time2)
        df.columns = ['start_ts', 'finish_ts', 'start_pos', 'finish_pos', 'target', 'success', 'user']
        if df.empty:
            print("There is no data for selected time and instrument")
            return ([],[])
        #df = format_df(bdf)
        #with open('../data/transfocator.txt', 'r') as handle:
        #df = pd.read_json(handle)
        #df = format_df(df)
        timeList = []
        positionList = []
        stoppedX = []
        stoppedY=[]        
        startX = []
        startY = []
        successX = []
        successY = []
        stopEvent = []
        for i in range(0, df.shape[0]):
            timeList.append((df['start_ts'][i]))
            timeList.append((df['finish_ts'][i]))
            positionList.append(df['start_pos'][i])
            positionList.append(df['finish_pos'][i])
            startX.append((df['start_ts'][i]))
            startY.append((df['start_pos'][i]))
            successX.append((df['finish_ts'][i]))
            successY.append(df['finish_pos'][i])
            if df['success'][i]=="False":
                stopInfo = unsuccess_line(df, stoppedX, stoppedY, i)
                stopEvent.append(stopInfo[0])
                stoppedX = stopInfo[1]
                stoppedY = stopInfo[2]
        event = go.Scatter(
                          x=timeList,
                          y=positionList,
                          hoverinfo = 'none',
                          showlegend = False,
                          line = dict(
                          color=motorColor
                          )
        )
        data=[]
        data = [event]
        #+bar_range(motorColor, df)
        if stoppedX!=[]:
            data = data+stopEvent+make_scatter(startX, startY, successX, successY, stoppedX, stoppedY, motorColor, num, motor_name)
        else:
            data = data+make_scatter(startX, startY, successX, successY, stoppedX, stoppedY, motorColor, num, motor_name)

        return [data, df]
#def format_df(df):
    #for i in range(0, df.shape[0]):
                #df.loc[i, 'start_ts'] = datetime.strptime(df['start_ts'][i].replace("T", " "), '%Y-%m-%d %H:%M:%S')
                #df.loc[i, 'finish_ts'] = datetime.strptime(df['finish_ts'][i].replace("T", " "), '%Y-%m-%d %H:%M:%S')
                #df.loc[i, 'start_ts'] = datetime.strptime(str(datetime.fromtimestamp(df['start_ts'][i])), "%Y-%m-%d %H:%M:%S")
                #df.loc[i, 'finish_ts'] = datetime.strptime(str(datetime.fromtimestamp(df['finish_ts'][i])), "%Y-%m-%d %H:%M:%S")
                #df.loc[i, 'target'] = int(round(df['target'][i]))
                #df.loc[i, 'finish_pos'] = int(round(df['finish_pos'][i]))

#if the target is not reached, the line is simulated and plotted as if it were reached as a red dashed line
def unsuccess_line(df, stoppedX, stoppedY, i):
    finish_time = datetime.strptime(df['finish_ts'][i].replace("T", " "), '%Y-%m-%d %H:%M:%S')
    start_time = datetime.strptime(df['start_ts'][i].replace("T", " "), '%Y-%m-%d %H:%M:%S')
    diff = float((finish_time-start_time).seconds)
    m = (df['finish_pos'][i]-df['start_pos'][i])/(diff)
    hypoTime = abs(round((df['target'][i]/m-df['finish_pos'][i]/m)))
    targetTime = timedelta(seconds = hypoTime)+finish_time
    stoppedX.append((targetTime))
    stoppedY.append(df['target'][i])
    stopEvent = go.Scatter(
        x=((df['start_ts'][i]), (targetTime)),
        y=(df['start_pos'][i], df['target'][i]),
        showlegend = False,
        hoverinfo = 'none',
        text = 'Hypothetical Complete Event',
        line = dict(
        dash = 'dash',
        color='red'
        )
    )
    return [stopEvent, stoppedX, stoppedY]
#creates the normal ranges for each motor to help troubleshoot if the motor stopped working accidentally
def bar_range(motorColor, df):
    startTime= df['start_ts'][0]
    endTime = df['finish_ts'][df.shape[0]-1]
    trace = go.Bar(
        name = "2018", orientation = "h",  
        y = [101], 
        x = [endTime], 
        #opacity = 0.6,
        hoverinfo = 'skip',
        width = 2,
        showlegend = False,
        #increasing = {"marker":{"color":motorColor}}, 
        
    )
    trace1 = go.Bar(
        name = "2018", orientation = "h",  
        y = [124], 
        x = [endTime], 
        #opacity = 0.6,
        hoverinfo = 'skip',
        width = 2,
        showlegend = False,
        #increasing = {"marker":{"color":motorColor}}, 
        
    )
    return([trace, trace1])



In [16]:
#def updatemenus():
#    menu = list([
#    dict(type="radiobuttons",
#         active = 1,
#         buttons=list([  
#            dict(label = 'Complete Table',
#                 method = 'update',
#                 args = [{'visible': [True]}]),
            #dict(label = 'Click Point for Info',
            #     method = 'update',
            #     args = [{'visible': [False]}]),
#    ]))])
#    return menu
#def tablelayout():
#    layout = go.Layout(
#    updatemenus = updatemenus(),
#    )
#    return layout

#sets the layout for the scatterplot in order to have the range slider and text
def set_layout(df, x):
    layout = go.Layout(showlegend = True,
    title=go.layout.Title(
        text='Motor Status',
        xref='paper',
        x=0.5,
        font = dict(
        size = 22)
    ),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='YTD',
                    step='year',
                    stepmode='todate'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Motor Position',
            font=dict(
                size=16,
            )
        )
    ),
  #  shapes = [dict
  #      (type = 'rect',
  #       x0= df['start_ts'][0],
  #       y0= 100,
  #       x1= df['finish_ts'][df.shape[0]-1],
  #       y1= 102,
  #      fillcolor= 'black',
  #      opacity = 0.1),
  #      dict
  #      (type = 'rect',
  #       x0= df['start_ts'][0],
  #       y0= 123,
  #       x1= df['finish_ts'][df.shape[0]-1],
  #       y1= 125,
  #      fillcolor= 'black',
  #      opacity = 0.1)],
                       
    #margin = dict(t=33)
                    )
    return layout

#sets the layout for the complete table and the single row table 
def table_layout(kind):
    if kind == 'tbl':
        layout = go.Layout(
        title=go.layout.Title(
        text='Complete Information Table',
        xref='paper',
        font=dict(
                size=15,
            ),
        x=0
        ),
        height = 300,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=30,
            t=30,
            pad=4
        ))
    else:
        layout = go.Layout(
        height = 130,
        title=go.layout.Title(
        text='Click on Start Point for Event Info',
        xref='paper',
        font=dict(
                size=15,
            ),
        x=0
        ),
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=10,
            t=30,
            pad=4
        ))
    return layout

#sets up the DropDown menu for motos adn the DatePickers for Start and End Date
motorList = ('AMO', 'CXI', 'MEC', 'MFX', 'SXR', 'XCS', 'XPP', 'fke', 'testStatus')
motors = widgets.SelectMultiple(options = motorList, value = ('testStatus',), description='Instrument(s)', disabled=False)
start_time = widgets.DatePicker(description='Start Date', disabled=False, value = datetime.strptime('2019-06-20', '%Y-%m-%d'))
stop_time = widgets.DatePicker(description='End Date', disabled=False, value = datetime.strptime('2019-06-20', '%Y-%m-%d'))

#called when motors are added or date is changed, uploads data into a plot and makes a table with the complete data 
def add_motors(motors, time1, time2):
    data=[]
    tables = []
    motorcolors = []
    oneFigure = go.Figure(layout = table_layout('one'))
    oneFig = go.FigureWidget(oneFigure)
    #display(oneFig)
    for i in range(0, len(motors)):
        colors = (ord(motors[i][:1]))*2,2*ord(motors[i][1:2]),2*(ord(motors[i][2:3]))
        motorColor = 'rgb'+str(colors)
        motorcolors.append(motorColor)
        data = data+plot_motor(motors[i], time1, time2, i)[0]
        if data==([]):
            return None
        tabledat = plot_motor(motors[i], time1, time2, i)[1]
        tables.append(tabledat)
        start_time = time1
        stop_time = time2

    layout = set_layout(tabledat, data[0].x)
    f = go.Figure(data=data, layout=layout)
    fig = go.FigureWidget(f)
    scatter = fig.data[0]
    display(fig)
    tableList = []
    tablefigs= []
    notext = ('  ', '   ', '   ')
    emptyTable = go.Table(header={'values': notext, 'line': dict(color='white'), 'height':5}, 
                          cells={'values': notext, 'font' : dict(color ='white'), 'line' : dict(color='white'), 'height':5})
    oneFig.add_trace(emptyTable)
    display(oneFig)
    #makes and displays complete tables of all plotted motors
    for i in range(0, len(tables)):
        df = tables[i]
        tableData = go.Table( header={'values': df.columns, 'fill' : dict(color=motorcolors[i])}, 
        cells={'values': [df[i] for i in df.columns]})
        tablesimpleFig = go.Figure(layout = table_layout('tbl'))
        tableFig = go.FigureWidget(tablesimpleFig)
        tableList.append(tableData)
        tableFig.add_trace(tableData)
        display(tableFig)
        tablefigs.append([tableFig])

    #for i in range(0, len(tablefigs)):
        
    #    tablefigs[i][0].layout = compLayout
    #    display(tablefigs[i][0])
        
    #print(tableFig)
    #compLayout = tablelayout(tableList)
    #tablesimpleFig.layout = compLayout
    #for i in range(0, len(tableList)):
    #    fig = dict(data = [tableList[i]])
    #    plotly.offline.iplot(fig)
    #onetablelayout = go.Layout(dict(margin = dict(t=10, b = 10)), updatemenus=updatemenus(tables))
    #tablesimpleFig = go.Figure(layout = compLayout)
    #tableFig = go.FigureWidget(tablesimpleFig)
    #for i in range(0, len(tableList)):
    #    tableFig.add_trace(tableList[i])
    #print(tableFig)
    #display(tableFig)

    #makes a single row table of a point when the point is clicked, this must be a start point
    def make_table(trace, points, state):
        #tableFig.data = []
        
        oneFig.data=[]
        for i in range(0, len(tables)):
            df = tables[i]
            for j in range(0, len(tables[i])):
                if (str(df.iloc[j]['start_ts']))==str(points.xs[0]):
                    #if(j>1 and j<len(tables[i])-2):
                    #    newdf = tables[i].iloc[(j-2):j+3].copy()
                        #for k in range(0, 5):
                            #newdf.iloc[k, 0] = str(newdf.iat[k, 0]).replace("T", " ")
                            #newdf.iloc[k, 1] = str(newdf.iat[k, 1]).replace("T", " ")
                    #else:
                        newdf = pd.DataFrame(columns = df.columns)
                        tbl = tables[i].iloc[j].copy()
                        list = []
                        for m in range(0, 7):
                            list.append(tbl[m])
                        newdf.loc[0] = list
                        #newdf.iloc[0, 0] = str(newdf.iat[0, 0]).replace("T", " ")
                        #newdf.iloc[0, 1] = str(newdf.iat[0, 1]).replace("T", " ")
                        tableData = go.Table(header={'values': newdf.columns, 'fill' : dict(color=motorcolors[i])}, 
                        cells={'values': [newdf[i] for i in newdf.columns]})
                        oneFig.add_trace(tableData)
                        #display(oneFig)
        if oneFig.data==():
            oneFig.add_trace(emptyTable)
                    
    scatter.on_click(make_table)

widgets.interactive(add_motors, motors= motors, time1=start_time, time2=stop_time)



interactive(children=(SelectMultiple(description='Instrument(s)', index=(8,), options=('AMO', 'CXI', 'MEC', 'M…